# Base de classification en Anglais 

    - Transformers : Summarization : 2 modèles --> 2 Résumés / Puis score de similarités de ces 2 résumés
    Noter que l'on peut faire aussi la similarité des textes (autre note ?) et non du résumé
    - Text classification sur une base de catégories "Presse" : sport - actaulités - économie - etc
    - Sentiment analysis : voir si le ton du texte est de même type 
    - Les 2 derniers classifier seronts utilisés en produit scalaire : Par Catégorie : texte1: note1 - texte2 : note2
    et donc sum(notes_par_catégorie) = sum(note1*note2) * 100 au bout (note sur 100)

In [1]:
import pandas as pd
import pke
import spacy
import torch
import stanza
import spacy_stanza
import warnings
import string
from gensim.models import KeyedVectors
import enchant    # Pour correction orthographique de synonymes
import numpy as np
import re
from transformers import pipeline
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelWithLMHead, AutoTokenizer
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
tqdm.pandas()
warnings.filterwarnings("ignore")

C:\Users\stg-sdu\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\stg-sdu\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\stg-sdu\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


**Selection des modèles NLP : ici ANGLAIS**

In [2]:
# Chargement pour l'utilisation de Spacy  - Français
nlp_en = spacy.load("en_core_web_sm")

In [3]:
dico_spacy = {'en':nlp_en}   # 'en':nlp_en,'de':nlp_de,'es':nlp_es,'pl':nlp_pl  - POUR MEMOIRE
langues = ['en','fr','es','de','pl','ar','tr']

In [4]:
# Chargement du modèle Word2Vec pour utilisation de synonymes
import gensim.downloader
model_gensim = gensim.downloader.load('glove-twitter-25')

In [5]:
# Stopwords Français NLTK + Spacy 
stopWords = list(nlp_en.Defaults.stop_words)
stopwords_en = list(stopwords.words('english'))  
stopwords_en = list(set(stopwords_en + stopWords))
stopwds_lg = {'en':stopwords_en}

In [6]:
# correcteur orthographique pour validation des synonymes OPTIONNEL CAR NON NECESSAIRE
d = enchant.Dict("en") 

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

**Sélection des modèles Transformers : Summary - Text Classification - Sentiment Analysis - Similarity**<br/>
    Pour le modèle anglais, on prend 3 summarizer !!!

In [8]:
# Modèles Transformers de Résumé (NB : Ne pas oublier d'ajouter la truncation pour tous les modèles, peut être source d'erreur)
summarizer1 = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", truncation = "only_first")   # ou sshleifer/distilbart-xsum-12-3
summarizer2 = pipeline("summarization", model="google/pegasus-xsum", truncation = "only_first")  # voir cnn 12-6

In [9]:
# Textes classification ou Zero shot classification (permet de chosir nos propres thèmes)
text_clf1 = pipeline("zero-shot-classification", model = "joeddav/bart-large-mnli-yahoo-answers", truncation = "only_first")   # 10 actégories, voir hugging face
text_clf2 = pipeline('zero-shot-classification', model='cross-encoder/nli-MiniLM2-L6-H768',truncation = "only_first")
# ces modèles sont zero shot classification : catégories possibles choisies par mes soins (dans la presse)
candidate_labels = ['Science', 'Politics', 'Education', 'News', 'Health', 'Technology', 'Society', 'Sport', 'Economy', 'Culture', 'International', 'Environment']

In [10]:
# Sentiment Analysis : base de 1 à 5 stars
sentiment1 = pipeline("text-classification", model = 'nlptown/bert-base-multilingual-uncased-sentiment',truncation = "only_first")

# Sur la base des sentiments classiques : joy, anger, suprise, sadness, love, fear
sentiment2 = pipeline("text-classification", model = 'bhadresh-savani/distilbert-base-uncased-emotion',truncation = "only_first")

# Sur la base des sentiments classiques : NEGATIVE / POSITIVE
sentiment3 = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english',truncation = "only_first")

In [11]:
# ENCODAGE AVEC SENTENCE TRANSFORMER 2 modfèles et moyenne
from sentence_transformers import SentenceTransformer,util
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
encoder2 = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
def score_similarite(sentence1,sentence2):
    # attention, pour que torch fonctionne en dimension sentence1 (et 2) est une liste simple
    embed1 = encoder.encode(sentence1, convert_to_tensor=True)
    embed2 = encoder.encode(sentence2, convert_to_tensor=True)
    embed3 = encoder2.encode(sentence1, convert_to_tensor=True)
    embed4 = encoder2.encode(sentence2, convert_to_tensor=True)
    return round(float(util.pytorch_cos_sim(embed1,embed2))+float(util.pytorch_cos_sim(embed3,embed4))*100/2,2)

**Selection Data par langues**

In [12]:
data = pd.read_csv('eval_data_prep_v1.csv')
# autres langues
# all_ang = pd.read_csv('eval_de_en_traduit.csv')

In [13]:
#it_it = pd.read_csv('eval_it_traduit_deepl.csv') 
ru_ru = pd.read_csv('allRussian.csv',sep=';')

In [16]:
ru_ru = ru_ru[['pair_id', 'pair_lang', 'title_1', 'text_1', 'title_2', 'text_2',]]

In [53]:
ru_ru.iloc[200:250,:]

,pair_id,pair_lang,title_1,text_1,title_2,text_2
200,1527282416_1527185438,ru_ru,Moscow uses facial recognition system because ...,"As of today, not a single case of coronavirus ...",To control the coronavirus quarantine in Mosco...,Moscow Mayor Sergei Sobyanin told in his blog ...
201,1618911364_1607739213,ru_ru,58.5 million rubles was additionally allocated...,"On May 28, 2020 at the meeting of the Legislat...",Dzerzhinsk checked the reconstruction of the C...,Dzerzhinsk checked the reconstruction of the C...
202,1625092898_1619794670,ru_ru,"In Yakutia, a recidivist morgue attendant stab...","Yakutsk, , 09:50 - REGNUM In the Vilyui distri...","In Yakutsk, a teenager burned in a garage","Yakutsk, , 05:18 - REGNUM In Yakutsk, investig..."
203,1518756489_1532089163,ru_ru,In Veliky Novgorod found ownerless objects wor...,"Veliky Novgorod, , 10:24 - REGNUM In Veliky No...",Mayor of Veliky Novgorod will report to the de...,"Veliky Novgorod, , 00:06 - REGNUM Mayor of Vel..."
204,1636460186_1636492918,ru_ru,The Ministry of Labor responded to the proposa...,Onishchenko responded to the statement of the ...,The Ministry of Labor received a draft to redu...,Photo: Kirill Zykov
205,1575309361_1574915744,ru_ru,OPEC+ has made a critical decision,Two days of hours of talks via video link and ...,G20 promises to ensure stability of the energy...,The G20 countries have agreed to create a grou...
206,1631652654_1639940005,ru_ru,The number of COVID-19 patients is growing aga...,"In China, where earlier a new version of the o...",China detects new cases of COVID-19: almost al...,From abroad only four new infected came to the...
207,1602074576_1602029357,ru_ru,Numerous congratulations on the 75th anniversa...,Numerous congratulations on the 75th anniversa...,The Head of State and the people of Belarus co...,"""The victory over Nazism is our common great ..."
208,1549725071_1578833577,ru_ru,What in the Murmansk region will be demolished...,"On key changes in the budget of the region, di...",In the Murmansk region the number of COVID-19 ...,In the Murmansk region the number of COVID-19 ...
209,1630949111_1551736260,ru_ru,In the Severodonetsk diocese of the UOC will b...,The site for the construction of a new temple ...,The Severodonetsk Eparchy of the UOC collects ...,In all churches and monasteries of the Severod...


In [49]:
ru_ru.text_2[149]

'MOSCOW, June 21. '

In [33]:
# es_en = pd.read_csv('eval_es_en_traduit.csv')
# it_it = pd.read_csv('eval_it_to_en_traduit.csv')   # traductions google deep translator
# ru_ru = pd.read_csv('eval_ru_to_en_traduit.csv')   # traductions google deep translator

In [17]:
# complément datasets polonais
# pl_pl = pd.read_csv('eval_pl_pl_traduit.csv')
# pl_en = pd.read_csv('eval_pl_en_traduit.csv')
# polonais = pd.concat([pl_pl,pl_en],axis=0).reset_index(drop=True)

In [18]:
# Attention, la première colonne ne correspond pas forcément à la ligne (a priori OK mais assurance)
# polonais = polonais.loc[:,polonais.columns[1:]]
# polonais = polonais.merge(data[['ligne','pair_id','pair_lang']],left_on = 'pair_id',right_on='pair_id')

In [19]:
# polonais

In [20]:
# remémorer numéro de ligne - compléter les Nan
# data['ligne'] = data.index
# data = data.fillna('')

In [21]:
# séparation des datasets, le dernier étant à traduire en plus
# anglais = data.loc[data.pair_lang == 'en_en',['ligne','title_1','title_2','text_1','text_2']].reset_index(drop=True)

In [114]:
# Fonction de calcul du score (produit scalaire) pour résultats de classifaction
def fonction_produit_dotcom(liste_categor, dico_scores1,dico_scores2):
    """"dico scores sont les résultats obtenus pour chaque catégorie des textes 1 et 2"""
    result = 0.0
    for cat in liste_categor:
        result += round(dico_scores1[cat] * dico_scores2[cat],4)
    return result * 100

In [115]:
# transformation des résultats du transformer type1
def transform_text_clf1(liste_dico):
    res = {}
    for dic in liste_dico:
        res[dic['label']] = dic['score']
    return res

In [116]:
# transformation des résultats du transformer type2
def transform_text_clf2(liste_cat,liste_sc):
    res = {}
    for i in range(len(liste_cat)):
        res[liste_cat[i]] = liste_sc[i]
    return res

In [117]:
# Tests sentiment analysis
liste_labels = ['1 star','2 stars','3 stars','4 stars','5 stars']
liste_sentiments = ['joy','anger','sadness','love','surprise','fear']
liste_sentiments2 = ['NEGATIVE','POSITIVE']

In [118]:
# Fonctions de summarization 
def summarization(texte):
    return summarizer1(texte)[0]['summary_text'], summarizer2(texte)[0]['summary_text']

In [119]:
dico_classifiers = {'text_clf1': 'score_classif1','text_clf2':'score_classif2','sentiment1':'score_sentiment1',
                    'sentiment2': 'score_sentiment2','sentiment3': 'score_sentiment3'}
dico_categories = {'text_clf1': candidate_labels,'text_clf2':candidate_labels,'sentiment1':liste_labels,
                    'sentiment2': liste_sentiments,'sentiment3': liste_sentiments2}

In [120]:
# Fonctions de classification et sentiment analysis
def classification(texte,clf):
    # assume nms des claasifiers et methode de transformation
    if clf == "text_clf1":
        try:
            classes = text_clf1(texte,dico_categories['text_clf1'])
        except:
            return 'error'
        else:
            return transform_text_clf2(classes['labels'],classes['scores'])    
    elif clf == "text_clf2":                                 
        try:
            classes = text_clf2(texte,dico_categories['text_clf2'])
        except:
            return 'error'
        else:
            return transform_text_clf2(classes['labels'],classes['scores'])                          
    elif clf == "sentiment1":
        try:
            scores = transform_text_clf1(sentiment1(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    elif clf == "sentiment2":
        try:
            scores = transform_text_clf1(sentiment2(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    elif clf == "sentiment3":
        try:
            scores = transform_text_clf1(sentiment3(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    else:
        return 'error'

In [121]:
# Prétraitement NLP pour PKE : suppression des mots de moins de 2 lettres non numériques
def supp_moins_2_lettres_stopwords(phrase,stopwd):
    temp = phrase.split(' ')
    res = ''
    for mot in temp:
        if mot not in stopwd and (len(mot)>2 or (len(mot)>0 and mot[0] in ['0','1','2','3','4','5','6','7','8','9'])):
            res += mot + ' '
    return res[:-1]

In [122]:
# Prétraitement NLP pour PKE : suppression des traits d'union(regroupe)/ des apostrophes / ponctuations
def modif(texte,stopmots):
    # modifications simples des textes : ponctuations, petits mots, stopwords (à faire pour entités et pke textes)
    texte=re.sub('\'',' ',texte)   # suppression apostrophe
    texte=re.sub('-','',texte)    # suppression trait union
    regex = re.compile('[%s]' % re.escape(string.punctuation)) # suppression de toutes les ponctuations
    texte=regex.sub(' ',texte)
    texte = supp_moins_2_lettres_stopwords(texte,stopmots)
    return texte

In [123]:
# Ajout des synonymes (existants en orthographe) à la suite de l'analyse pke
def ajout_synonymes(mot, correct_ortho = True):
    # on ajoute les 10 premiers synonymes existants, on vérifie orthographe (optionnel)
    syns = model_gensim.most_similar(mot,topn = 20)
    if correct_ortho == True:
        res = []
        for m in syns:
            if d.check(m[0]):   #  il y a le mot et son pourcentage d'importance
                res.append(m)
        syns = res
    return syns[:10]

In [124]:
# Choix des paramètres de la méthode : A revoir ?
methode1 = {"NOUN", "PROPN", "ADJ","VERB"}
methode2 = {"NOUN", "PROPN", "ADJ"}
nb_mots = {'meth1': 30, 'meth2':50}

In [125]:
# PKE : Analyse des termes principaux dans les textes et titres 
# Problème 
def transformation_pke_results(res1,res2, correct_ortho = True):
    """
    Transformation des resultats de PKE : Pb bigramme peuvent ne pas être ds les 2 textes mais 1 mot seulement
    liste de clés et dictionnaires de valeurs, bigrammes jouera ainsi de maniere coefficientée 
    Exemple : fuite eau:0.05 --> 3 mots au final : fuite, eau, fuite eau : 0.05
    De plus on ajoute les synonymes issus de gensim en les coefficiant et vérifiant que cela """
    
    liste1 = []; liste2 = [] ; dico1 = {}; dico2 = {}
    for elt in res1:
        liste1.append(elt[0])
        dico1[elt[0]] = round(elt[1],3)
        if ' ' in elt[0]:    # bigramme dans ce cas, ajout des 2 mots
            liste = elt[0].split(' ')
            for mot in liste:
                liste1.append(mot)
                dico1[mot] = round(elt[1],3)
                try:
                    synonyms = ajout_synonymes(mot,correct_ortho = correct_ortho)
                except:
                    pass
                else:
                    for syn in synonyms:
                        liste1.append(syn[0])   # Ajout du mot 
                        dico1[syn[0]] = round(elt[1] * syn[1], 3)  # poids considéré
                    
    for elt in res2:
        liste2.append(elt[0])
        dico2[elt[0]] = round(elt[1],3)
        if ' ' in elt[0]:
            liste = elt[0].split(' ')
            for mot in liste:
                liste2.append(mot)
                dico2[mot] = round(elt[1],3)
                try:
                    synonyms = ajout_synonymes(mot,correct_ortho = correct_ortho)
                except:
                    pass
                else:
                    for syn in synonyms:
                        liste2.append(syn[0])   # Ajout du mot 
                        dico2[syn[0]] = round(elt[1] * syn[1], 3)  # poids considéré
    
    # similarites entre les 2 listes issus de pke avec poids
    sim = 0
    for elt in liste1:
        if elt in liste2:
            sim += (dico1[elt] + dico2[elt])/2
    return sim

In [126]:
def entites_communes(nlp,text1,text2):
    """"
    Cette première fonction ne regarde que les entités communes : personnes, dates, groupe, localisations
    Elle sera appliquée aux textes et aux titres et cumulé : si cumul en titre et texte : compte double !"""
    
    try:
        doc1 = nlp(text1)
        doc2 = nlp(text2)
    except:
        return (0,[],0,[],0,[])
    else:
        nb_commun_ent = 0; liste_commun_ent = []
        nb_commun_geo = 0; liste_commun_geo = []
        nb_commun_dat = 0; liste_commun_dat = []

        if len(doc1.ents)>0 and len(doc2.ents)>0:
            liste1 = []; dico1 = {}
            for elt in doc1.ents:
                if elt.label_ in ['PERSON','PER'] and ' ' in elt.text:
                    mots = elt.text.split(' ')
                    for mot in mots:
                        if mot not in liste1:
                            liste1.append(mot)
                            dico1[mot] = elt.label_
                elif elt.label_ in ['LOC','ORG','GPE','DATE','TIME']:
                    if elt.text not in liste1:
                        liste1.append(elt.text)
                        dico1[elt.text] = elt.label_
            liste2 = []
            for elt in doc2.ents:
                if elt.label_ in ['PERSON','PER'] and ' ' in elt.text:
                    mots = elt.text.split(' ')
                    for mot in mots:
                        if mot not in liste2:
                            liste2.append(mot)
                elif elt.label_ in ['LOC','ORG','GPE','DATE','TIME']:
                    if elt.text not in liste2:
                        liste2.append(elt.text)

            # points communs des listes        
            for elt in liste1:
                if elt in liste2:
                    if dico1[elt] == 'LOC':
                        nb_commun_geo += 1
                        liste_commun_geo.append(elt)
                    elif dico1[elt] in ['DATE','TIME']:
                        nb_commun_dat += 1
                        liste_commun_dat.append(elt)
                    else:
                        nb_commun_ent += 1
                        liste_commun_ent.append(elt)

        return nb_commun_ent, liste_commun_ent,nb_commun_geo, liste_commun_geo,nb_commun_dat, liste_commun_dat

In [127]:
def Creation_features_comparaison(df,langue, test_position = [methode1,methode2]):
    """Création des notes pour classification ensuite"""
    
    resultats = pd.DataFrame(columns = ['summary1_text1','summary2_text1','summary1_text2','summary2_text2',
            'nb_entites_idem','nb_lieux_idem', 'nb_dates_idem','entites_idem','lieux_idem','dates_idem',
            'score_similarite_titres','score_similarite_resume1','score_similarite_resume2','score_classif1','score_classif2',
            'score_sentiment1','score_sentiment2','score_sentiment3','meth1_similarites','meth2_similarites'])
    
    # initialisation de la langue stanza
    stanza.download(langue)
    nlp_stanza = spacy_stanza.load_pipeline(langue)
    stopmts = stopwds_lg[langue]
    if langue in dico_spacy.keys():
        nlp_spacy = dico_spacy[langue]
    else:
        nlp_spacy = None
        
    for i in tqdm(range(len(df))):
        dico_res = {}
        
        # Summary et comparatifs 
        try:
            dico_res['summary1_text1'],dico_res['summary2_text1'] = summarization(df.text_1[i])
        except:
            dico_res['summary1_text1'] = '';dico_res['summary2_text1'] = ''
        try:
            dico_res['summary1_text2'],dico_res['summary2_text2'] = summarization(df.text_2[i])
        except:
            dico_res['summary1_text2'] = '';dico_res['summary2_text2'] = ''
        try:
            dico_res['score_similarite_titres'] = score_similarite(df.title_1[i],df.title_2[i])
        except:
            dico_res['score_similarite_titres'] = 'Error'
        if len(dico_res['summary1_text1'])>0 and len(dico_res['summary1_text2'])>0:
            dico_res['score_similarite_resume1'] = score_similarite(dico_res['summary1_text1'],dico_res['summary1_text2'])
        else:
            dico_res['score_similarite_resume1'] = 0
        if len(dico_res['summary2_text1'])>0 and len(dico_res['summary2_text2'])>0:
            dico_res['score_similarite_resume2'] = score_similarite(dico_res['summary2_text1'],dico_res['summary2_text2'])
        else:
            dico_res['score_similarite_resume2'] = 0
        
        # analyse de textes classification et de sentiments
        if type(df.title_1[i]) == str and type(df.text_1[i]) == str:
            texte1 = df.title_1[i] + ' ' + df.text_1[i]
        elif type(df.title_1[i]) == str:
            texte1 = df.title_1[i]
        elif type(df.text_1[i]) == str:
            texte1 = df.text_1[i]   
        else:
            texte1=''
        if type(df.title_2[i]) == str and type(df.text_2[i]) == str:
            texte2 = df.title_2[i] + ' ' + df.text_2[i]
        elif type(df.title_2[i]) == str:
            texte2 = df.title_2[i]
        elif type(df.text_2[i]) == str:
            texte2 = df.text_2[i]   
        else:
            texte2=''
        if len(texte1)>0 and len(texte2)>0:
            for classifier in dico_classifiers.keys():
                scores1 = classification(texte1,classifier)
                scores2 = classification(texte2,classifier)
                if scores1 != 'error' and scores2 != 'error':
                    dico_res[dico_classifiers[classifier]] = fonction_produit_dotcom(dico_categories[classifier], scores1,scores2)
                else:
                    scores1 = classification(df.title_1[i],classifier)
                    scores2 = classification(df.title_2[i],classifier)
                    if scores1 != 'error' and scores2 != 'error':
                        dico_res[dico_classifiers[classifier]] = fonction_produit_dotcom(dico_categories[classifier], scores1,scores2)
                    else:
                        dico_res[dico_classifiers[classifier]] = None
        else:
            for classifier in dico_classifiers.keys():
                dico_res[dico_classifiers[classifier]] = 0
                
        # pré traitement des textes pour entités et PKE
        texte1 = modif(texte1, stopmts)
        texte2 = modif(texte2, stopmts)
        
        # ENTITES COMMUNES : on tient compte des bigrammes Noms qui posent erreurs ex: Joe Biden et Biden 
        # Ici, on considère mieux le CUMUl titres et Textes avec une pondération double pour le titre 
        # Il faut aussi enlever les petits mots donc pré-traitement en texte
        
        try:
            nb_ent1,list_ent1,nb_geo1,list_geo1,nb_dat1,list_dat1 = entites_communes(nlp_stanza,df.title_1[i],df.title_2[i])
        except:
            nb_ent1,list_ent1,nb_geo1,list_geo1,nb_dat1,list_dat1 = (0,[],0,[],0,[])
        try:
            nb_ent2,list_ent2,nb_geo2,list_geo2,nb_dat2,list_dat2 = entites_communes(nlp_stanza,df.text_1[i],df.text_2[i])
        except:
            nb_ent2,list_ent2,nb_geo2,list_geo2,nb_dat2,list_dat2 = (0,[],0,[],0,[])                                                                       
        if nlp_spacy != None:
            nb_ent3,list_ent3,nb_geo3,list_geo3,nb_dat3,list_dat3 = entites_communes(nlp_spacy,df.title_1[i],df.title_2[i])
            nb_ent4,list_ent4,nb_geo4,list_geo4,nb_dat4,list_dat4 = entites_communes(nlp_spacy,df.text_1[i],df.text_2[i])
        else:
            nb_ent3,list_ent3,nb_geo3,list_geo3,nb_dat3,list_dat3 = (0,[],0,[],0,[])
            nb_ent4,list_ent4,nb_geo4,list_geo4,nb_dat4,list_dat4 = (0,[],0,[],0,[])
        dico_res['nb_entites_idem'] = nb_ent1 * 2 + nb_ent2 + nb_ent3 * 2 + nb_ent4
        dico_res['nb_lieux_idem'] = nb_geo1  * 2 + nb_geo2 + nb_geo3  * 2 + nb_geo4
        dico_res['nb_dates_idem'] = nb_dat1 * 2 + nb_dat2 + nb_dat3 * 2 + nb_dat4
        # fusion des listes en supprimant les doublons
        dico_res['entites_idem'] = list(set(list_ent1+list_ent2+ list_ent3+list_ent4))
        dico_res['lieux_idem'] = list(set(list_geo1+list_geo2+list_geo3+list_geo4))
        dico_res['dates_idem'] = list(set(list_dat1+list_dat2+list_dat3+list_dat4))
        
        for j,meth in enumerate(test_position):
            nom ='meth'+str(j+1)
            nb_mots_meth = nb_mots[nom]
            if len(texte1)>0 and len(texte2)>0:
                extractor = pke.unsupervised.TopicRank()
                extractor.load_document(input=texte1,language=langue,normalization="stemming")
                extractor.candidate_selection(pos=meth)
                extractor.candidate_weighting()
                keyphrases3 = extractor.get_n_best(n=nb_mots_meth)
                extractor = pke.unsupervised.TopicRank()
                extractor.load_document(input=texte2,language=langue,normalization="stemming")
                extractor.candidate_selection(pos=meth)
                extractor.candidate_weighting()
                keyphrases4 = extractor.get_n_best(n=nb_mots_meth)
                dico_res[nom+'_similarites'] = round(100*transformation_pke_results(keyphrases3,keyphrases4),1)
            else:
                dico_res[nom+'_similarites'] = 'Error'
        
        resultats.loc[len(resultats)] = dico_res
        
    newdf = pd.concat([df,resultats],axis=1)
    return newdf

In [128]:
# similarites = Creation_features_comparaison(anglais.reset_index(drop=True),'en')
# similarites = Creation_features_comparaison(all_ang.reset_index(drop=True),'en')
# similarites = Creation_features_comparaison(polonais.reset_index(drop=True),'en')
# similarites = Creation_features_comparaison(es_en[:250].reset_index(drop=True),'en')
similarites = Creation_features_comparaison(it_it.reset_index(drop=True),'en')
#similarites = Creation_features_comparaison(ru_ru.reset_index(drop=True),'en')

2022-01-23 18:23:03 INFO: Downloading default packages for language: en (English)...
2022-01-23 18:23:04 INFO: File exists: C:\Users\stg-sdu\stanza_resources\en\default.zip.
2022-01-23 18:23:07 INFO: Finished downloading models and saved to C:\Users\stg-sdu\stanza_resources.
2022-01-23 18:23:07 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-01-23 18:23:07 INFO: Use device: cpu
2022-01-23 18:23:07 INFO: Loading: tokenize
2022-01-23 18:23:07 INFO: Loading: pos
2022-01-23 18:23:07 INFO: Loading: lemma
2022-01-23 18:23:07 INFO: Loading: depparse
2022-01-23 18:23:08 INFO: Loading: sentiment
2022-01-23 18:23:08 INFO: Loading: constituency
2022-01-23 18:23:08 INFO: Loading: ner
2022-01-23 18:23:09 INFO: Done loading proces

  0%|          | 0/442 [00:00<?, ?it/s]

Your max_length is set to 142, but you input_length is only 125. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 76. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 99. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 134. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 85. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 97. You might consi

Your max_length is set to 142, but you input_length is only 141. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 82. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 93. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 63. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 93. You might conside

In [130]:
similarites

,pair_id,pair_lang,title_1,text_1,title_2,text_2,ligne,summary1_text1,summary2_text1,summary1_text2,...,score_similarite_titres,score_similarite_resume1,score_similarite_resume2,score_classif1,score_classif2,score_sentiment1,score_sentiment2,score_sentiment3,meth1_similarites,meth2_similarites
0,1573959882_1642941640,it_it,"Fire in the attic of a villa in Monfalcone, fi...",MONFALCONE. Firefighters at work in via San Po...,"Monfalcone, elderly woman hit by a car: she is...",MONFALCONE. A serious car accident occurred to...,1104,Firefighters at work in via San Polo 1 in Mon...,Firefighters at work in via San Polo 1 in Monf...,A Citroen Picasso hit an elderly woman who wa...,...,11.43,11.21,6.24,9.54,8.86,16.04,37.96,95.73,0,190.6
1,1560404523_1545160678,it_it,"GdS - Uefa and Fifa study the new calendar, bu...",The virus is in charge and the resumption of a...,"The Lega di Serie A: ""Clubs that do not play i...","The communication on the official website: ""Pl...",1105,The virus is in charge and the resumption of ...,The start of the Serie A season is in doubt be...,"Lega Serie A observes ""in consideration of th...",...,28.23,28.52,15.43,12.72,9.66,21.79,25.86,80.33,0,472.1
2,1504994205_1533599163,it_it,"Tourism in Grottaglie, there is a sold-out","GROTTAGLIE - ""Tourism, there is an increase of...","Coronavirus, in Veneto the first victim. 19 ca...",It 's resident in Sesto Cremonese the 16th Lom...,1106,Grottaglie tourist information point has exte...,Grottaglie closes the Christmas period with a ...,Man in Sesto Cremonese is the 16th Lombard fo...,...,6.54,8.25,7.02,4.07,7.75,18.07,1.21,1.07,23.4,470.7
3,1599896557_1549928647,it_it,"Let's open the schools, but on one condition. ...","Then, let's start from an assumption, which I ...","""Need a break for workers"". Coop announces sto...",Coop has decided to close its 1100+ stores on ...,1107,"At this moment, the only really effective saf...","Let's start from an assumption, which I believ...",Coop will close its 1100+ stores on Sundays f...,...,12.10,1.73,-0.57,6.34,8.44,23.41,21.40,91.84,398.6,523.8
4,1571459332_1610838005,it_it,"Padova, fire devastates the Euganean Hills",PADOVA A high column of white smoke visible th...,"Padua, house fire: wife of pharmacist from Aba...",The alarm was triggered Saturday evening short...,1108,A fire broke out around 1:30 p.m. in the Euga...,A forest fire that broke out in the Euganean H...,The alarm was triggered Saturday evening shor...,...,18.61,28.61,8.21,10.53,8.42,17.66,45.10,91.98,254.9,229.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,1544911607_1544697909,it_it,"Coronavirus, Rixi: ""No half-measures: total cl...","La Spezia - ""The total closure of all activiti...","Coronavirus, Rixi (League): ""No half measures....","Region. ""The total closure of all activities n...",2659,The League deputy of the League Edoardo Rixi ...,"The deputy of the League, Edoardo Rixi, has ca...","Lega deputy: ""The total closure of all activi...",...,48.01,34.37,46.30,13.14,8.70,39.29,47.05,98.87,1564.7,1268.8
438,1558669509_1558594807,it_it,"Influenza, infections in rapid decline but so ...","- L'""Io resto a casa"" imposto dall'emergenza c...","Influenza, 264 thousand cases in 7 days: incid...",Photogram / Ipa\n\nIn the week from 9 to 15 Ma...,2660,"L'""Io resto a casa"" imposto dall'emergenza co...",La settimana precedente di sorveglianza epidem...,The number of cases of flu-like syndrome cont...,...,43.54,33.01,21.22,63.09,8.23,29.67,20.82,97.40,11,4.5
439,1533800341_1540033772,it_it,"Agreement with the house of Mickey Mouse, Disn...","At any rate, the arrival of Disney+ represents...",Disney+ arrives in Italy exclusively on TimVision,"The content of Disney+, Disney's streaming pla...",2661,The arrival of Disney+ represents the cleares...,The arrival of Disney+ on Timvision's video-on...,"Disney's streaming platform, Disney+, arrives...",...,28.87,28.61,15.89,14.08,8.72,25.92,93.84,1.61,597.1,511.4
440,16

In [ ]:
#similarites.to_csv('eval_en_notes.csv') 
#similarites.to_csv('eval_de_en_notes.csv') 
# similarites.to_csv('eval_pl_notes.csv') 
# similarites.to_csv('eval_es_en_notes.csv') 
similarites.to_csv('eval_it_en_notes_deepl.csv')
# similarites.to_csv('eval_ru_en_notes.csv')

In [33]:
# similarites = Creation_features_comparaison(es_en[300:].reset_index(drop=True),'en')
# precedent = pd.read_csv('eval_es_en_notes.csv',index_col=0)
# similarites2 = pd.concat([precedent,similarites], axis=0)
# similarites2 = similarites2.reset_index(drop=True)
# similarites2.to_csv('eval_es_en_notes.csv')

**Autres langues**

In [187]:
chinois = pd.read_csv('chinois_traduit.csv')

In [190]:
# les 600 premires lignes en 2 coups
similarites = Creation_features_comparaison(chinois[:300].reset_index(drop=True),'en')

2022-01-14 21:23:16 INFO: Downloading default packages for language: en (English)...
2022-01-14 21:23:17 INFO: File exists: C:\Users\stg-sdu\stanza_resources\en\default.zip.
2022-01-14 21:23:20 INFO: Finished downloading models and saved to C:\Users\stg-sdu\stanza_resources.
2022-01-14 21:23:20 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-01-14 21:23:20 INFO: Use device: cpu
2022-01-14 21:23:20 INFO: Loading: tokenize
2022-01-14 21:23:20 INFO: Loading: pos
2022-01-14 21:23:20 INFO: Loading: lemma
2022-01-14 21:23:20 INFO: Loading: depparse
2022-01-14 21:23:20 INFO: Loading: sentiment
2022-01-14 21:23:20 INFO: Loading: constituency
2022-01-14 21:23:21 INFO: Loading: ner
2022-01-14 21:23:21 INFO: Done loading proces

  0%|          | 0/300 [00:00<?, ?it/s]

Your max_length is set to 142, but you input_length is only 133. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 23. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 74. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 61. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 102. You might consider 

Your max_length is set to 142, but you input_length is only 5. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 120. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 88. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 99. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 117. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 25. You might consider d

Your max_length is set to 64, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 93. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 99. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 64. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [191]:
similarites.to_csv('eval_zh_notes.csv') 

In [193]:
similarites = Creation_features_comparaison(chinois[300:600].reset_index(drop=True),'en')
precedent = pd.read_csv('eval_zh_notes.csv',index_col=0)
similarites2 = pd.concat([precedent,similarites], axis=0)
similarites2 = similarites2.reset_index(drop=True)
similarites2.to_csv('eval_zh_notes.csv')

2022-01-16 00:41:08 INFO: Downloading default packages for language: en (English)...
2022-01-16 00:41:09 INFO: File exists: C:\Users\stg-sdu\stanza_resources\en\default.zip.
2022-01-16 00:41:12 INFO: Finished downloading models and saved to C:\Users\stg-sdu\stanza_resources.
2022-01-16 00:41:12 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-01-16 00:41:12 INFO: Use device: cpu
2022-01-16 00:41:12 INFO: Loading: tokenize
2022-01-16 00:41:12 INFO: Loading: pos
2022-01-16 00:41:13 INFO: Loading: lemma
2022-01-16 00:41:13 INFO: Loading: depparse
2022-01-16 00:41:13 INFO: Loading: sentiment
2022-01-16 00:41:13 INFO: Loading: constituency
2022-01-16 00:41:14 INFO: Loading: ner
2022-01-16 00:41:14 INFO: Done loading proces

  0%|          | 0/300 [00:00<?, ?it/s]

Your max_length is set to 142, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 24. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 21. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 15. You might consider dec

Your max_length is set to 142, but you input_length is only 102. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 124. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 89. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 115. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 77. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 128. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 133. You might con

Your max_length is set to 142, but you input_length is only 110. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 87. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 87. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 29. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 18. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 18. You might consider 

In [30]:
chinois = pd.read_csv('chinois_traduit.csv')

In [31]:
similarites = Creation_features_comparaison(chinois[600:].reset_index(drop=True),'en')
precedent = pd.read_csv('eval_zh_notes.csv',index_col=0)
similarites2 = pd.concat([precedent,similarites], axis=0)
similarites2 = similarites2.reset_index(drop=True)
similarites2.to_csv('eval_zh_notes.csv')

2022-01-16 13:02:04 INFO: Downloading default packages for language: en (English)...
2022-01-16 13:02:05 INFO: File exists: C:\Users\stg-sdu\stanza_resources\en\default.zip.
2022-01-16 13:02:08 INFO: Finished downloading models and saved to C:\Users\stg-sdu\stanza_resources.
2022-01-16 13:02:08 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-01-16 13:02:08 INFO: Use device: cpu
2022-01-16 13:02:08 INFO: Loading: tokenize
2022-01-16 13:02:08 INFO: Loading: pos
2022-01-16 13:02:08 INFO: Loading: lemma
2022-01-16 13:02:08 INFO: Loading: depparse
2022-01-16 13:02:08 INFO: Loading: sentiment
2022-01-16 13:02:09 INFO: Loading: constituency
2022-01-16 13:02:09 INFO: Loading: ner
2022-01-16 13:02:10 INFO: Done loading proces

  0%|          | 0/169 [00:00<?, ?it/s]

Your max_length is set to 142, but you input_length is only 100. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 101. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 10. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 10. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 9. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 17. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 17. You might consider d

Your max_length is set to 142, but you input_length is only 92. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 31. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 26. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 27. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 20. You might consider dec